# 03 - Despliegue del Servicio Web en Azure
---
Este notebook contiene el despliegue de un modelo de regresión como API usando **Flask**. 

### Importamos Librerías

In [8]:
from flask import Flask, request, jsonify
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import joblib
import numpy as np
import json

### Creación de Workspace y Registro de Modelo

In [9]:
# Cargando el modelo
id = open('id_confidential.json', 'r')
mi = json.load(id)
my_id = mi["azure_id"]

ws = Workspace.create(name="sales",
                      subscription_id = my_id,
                      resource_group = "salesservice",
                      location = "centralindia")

registered_model = Model.register(model_path="model.pkl",
                                  model_name="modifiedate_model",
                                  workspace=ws)

Deploying KeyVault with name saleskeyvault1a042d3e8e4.
Deploying AppInsights with name salesinsightsa2d97326be4.
Deployed AppInsights with name salesinsightsa2d97326be4. Took 9.15 seconds.
Deploying StorageAccount with name salesstorageb2d2f0957d56.
Deployed KeyVault with name saleskeyvault1a042d3e8e4. Took 26.27 seconds.
Deploying Workspace with name sales.
Deployed StorageAccount with name salesstorageb2d2f0957d56. Took 29.31 seconds.
Deployed Workspace with name sales. Took 39.29 seconds.
Registering model modifiedate_model


### Creación del ScorePY

In [10]:
scorepy = """import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('modifiedate_model')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['datos'][0]
        data = pd.DataFrame(data)

        # Seleccionar columnas relevantes
        data = data[['CompanyName', 'SalesPerson', 'ModifiedDate']]

        # Convertir ModifiedDate a datetime y luego a rango de días
        data['ModifiedDate'] = pd.to_datetime(data['ModifiedDate'])
        data['Fecha'] = (data['ModifiedDate'] - data['ModifiedDate'].min()).dt.days

        # Eliminar columna original
        data.drop(columns=['ModifiedDate'], inplace=True)

        # Realizar predicción usando solo la columna transformada
        result = model.predict(data[['Fecha']])

        return json.dumps({"result": result.tolist()})
    except Exception as e:
        return json.dumps({"error": str(e)})
"""

with open("score.py", "w") as file_score:
    file_score.write(scorepy)

### Configuración de Inferencia

In [11]:
virtual_env = Environment("env_sales")

virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn'])

inference_config = InferenceConfig(environment=virtual_env,
                                   entry_script="score.py")

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=8)

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\blast\AppData\Local\Temp\ipykernel_13664\2385395119.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [12]:
service.wait_for_deployment()

Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 5b36bdc4-9c71-4c5d-b8c0-b1bc502c9dde
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config.",
  "details": [
    {
      "code": "ScoreInitRestart",
      "message": "Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 5b36bdc4-9c71-4c5d-b8c0-b1bc502c9dde
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config.",
  "details": [
    {
      "code": "ScoreInitRestart",
      "message": "Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: 5b36bdc4-9c71-4c5d-b8c0-b1bc502c9dde\nMore information can be found using

### Creación del URI

In [ ]:
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

In [ ]:
print(scoring_uri)